In [21]:
import re
from dataclasses import dataclass, field


@dataclass
class CompState:
    A: int
    B: int
    C: int
    program: list[str | int]
    instruction_pointer: int = 0
    output: list[int] = field(default_factory=list)

    def combo(self, operand: int):
        if operand <= 3:
            return operand
        elif operand == 4:
            return self.A
        elif operand == 5:
            return self.B
        elif operand == 6:
            return self.C

    def replace_opcodes(self):
        res = []
        OPCODES = ["adv", "bxl", "bst", "jnz", "bxc", "out", "bdv", "cdv"]
        for i, ins in enumerate(self.program):
            if i % 2 == 0:
                res.append(OPCODES[int(ins)])
            else:
                res.append(int(ins))

        self.program = res

    def adv(self, b):
        self.A = int(self.A / pow(2, self.combo(b)))

    def bdv(self, b):
        self.B = int(self.A / pow(2, self.combo(b)))

    def cdv(self, b):
        self.C = int(self.A / pow(2, self.combo(b)))

    def bxl(self, b):
        self.B = self.B ^ b

    def bst(self, b):
        self.B = self.combo(b) % 8

    def bxc(self, b):
        self.B = self.B ^ self.C

    def out(self, b):
        self.output.append(self.combo(b) % 8)

    def jnz(self, b):
        if self.A == 0:
            pass
        else:
            self.instruction_pointer = b

    def execute_current_command(self):
        opcode, operand = (
            self.program[self.instruction_pointer],
            self.program[self.instruction_pointer + 1],
        )

        if opcode == "jnz" and self.A != 0:
            pass
        else:
            self.instruction_pointer += 2

        getattr(self, opcode)(operand)

    def execute_program(self, debug=False):
        while self.instruction_pointer < len(self.program):
            self.execute_current_command()
            if debug:
                print(self)
        return ",".join([str(x) for x in self.output])


def read_file(path):
    res = open(path, "r").readlines()
    res = [x.strip() for x in res]
    for i, l in enumerate(res):
        if l.startswith("Register"):
            res[i] = re.sub("Register .*: ", "", l)
        elif l.startswith("Program"):
            res[i] = re.sub("Program: ", "", l)
    res = CompState(int(res[0]), int(res[1]), int(res[2]), res[4].split(","))
    res.replace_opcodes()
    return res

In [15]:
c = read_file("input")
c.execute_program()

'7,3,5,7,5,7,4,3,0'

In [22]:
c = read_file("example2")
c.execute_program(debug=True)

CompState(A=14680, B=0, C=0, program=['adv', 3, 'out', 4, 'jnz', 0], instruction_pointer=2, output=[])
CompState(A=14680, B=0, C=0, program=['adv', 3, 'out', 4, 'jnz', 0], instruction_pointer=4, output=[0])
CompState(A=14680, B=0, C=0, program=['adv', 3, 'out', 4, 'jnz', 0], instruction_pointer=0, output=[0])
CompState(A=1835, B=0, C=0, program=['adv', 3, 'out', 4, 'jnz', 0], instruction_pointer=2, output=[0])
CompState(A=1835, B=0, C=0, program=['adv', 3, 'out', 4, 'jnz', 0], instruction_pointer=4, output=[0, 3])
CompState(A=1835, B=0, C=0, program=['adv', 3, 'out', 4, 'jnz', 0], instruction_pointer=0, output=[0, 3])
CompState(A=229, B=0, C=0, program=['adv', 3, 'out', 4, 'jnz', 0], instruction_pointer=2, output=[0, 3])
CompState(A=229, B=0, C=0, program=['adv', 3, 'out', 4, 'jnz', 0], instruction_pointer=4, output=[0, 3, 5])
CompState(A=229, B=0, C=0, program=['adv', 3, 'out', 4, 'jnz', 0], instruction_pointer=0, output=[0, 3, 5])
CompState(A=28, B=0, C=0, program=['adv', 3, 'out', 4

'0,3,5,4,3,0'

In [48]:
c = read_file("example2")
c.execute_program()
needed_digits = list(reversed(c.output))
print(needed_digits)

regA = 0
for i, digit in enumerate(needed_digits):
    regA = (regA * 8) + digit
    print(regA)

2
20
161
1293
10351
82813
662508
5300067
42400537
339204302
2713634416
21709075331
173672602653
1389380821229
11115046569835
88920372558680


In [75]:
c = read_file("input")
c.execute_program()
c.output

[3, 2]

In [82]:
def check_me(a):
    b = a % 8
    b = b ^ 5
    c = int(a / pow(2, b))
    b = b ^ c
    b = b ^ 6
    a = a / pow(2, 3)
    return b % 8


def solve(program, ans):
    print(program, ans)
    if program == []:
        return ans
    for t in range(8):
        a = ans << 3 | t
        b = a % 8
        b = b ^ 5
        c = a >> b
        b = b ^ c
        b = b ^ 6
        if b % 8 == program[-1]:
            sub = solve(program[:-1], a)
            if sub is None:
                continue
            return sub


print(solve([2, 4, 1, 5, 7, 5, 4, 3, 1, 6, 0, 3, 5, 5, 3, 0], 0))

[2, 4, 1, 5, 7, 5, 4, 3, 1, 6, 0, 3, 5, 5, 3, 0] 0
[2, 4, 1, 5, 7, 5, 4, 3, 1, 6, 0, 3, 5, 5, 3] 3
[2, 4, 1, 5, 7, 5, 4, 3, 1, 6, 0, 3, 5, 5] 24
[2, 4, 1, 5, 7, 5, 4, 3, 1, 6, 0, 3, 5] 192
[2, 4, 1, 5, 7, 5, 4, 3, 1, 6, 0, 3] 1538
[2, 4, 1, 5, 7, 5, 4, 3, 1, 6, 0] 12304
[2, 4, 1, 5, 7, 5, 4, 3, 1, 6] 98434
[2, 4, 1, 5, 7, 5, 4, 3, 1, 6, 0] 12305
[2, 4, 1, 5, 7, 5, 4, 3, 1, 6] 98442
[2, 4, 1, 5, 7, 5, 4, 3, 1, 6, 0] 12309
[2, 4, 1, 5, 7, 5, 4, 3, 1, 6] 98473
[2, 4, 1, 5, 7, 5, 4, 3, 1] 787785
[2, 4, 1, 5, 7, 5, 4, 3] 6302280
[2, 4, 1, 5, 7, 5, 4] 50418245
[2, 4, 1, 5, 7, 5, 4, 3] 6302283
[2, 4, 1, 5, 7, 5, 4] 50418269
[2, 4, 1, 5, 7, 5] 403346152
[2, 4, 1, 5, 7] 3226769219
[2, 4, 1, 5, 7] 3226769220
[2, 4, 1, 5, 7] 3226769222
[2, 4, 1, 5, 7] 3226769223
[2, 4, 1, 5, 7, 5] 403346153
[2, 4, 1, 5, 7] 3226769227
[2, 4, 1, 5] 25814153817
[2, 4, 1] 206513230536
[2, 4] 1652105844288
[2, 4] 1652105844291
[2, 4, 1, 5, 7, 5] 403346154
[2, 4, 1, 5, 7] 3226769235
[2, 4, 1, 5] 25814153880
[2, 4, 1] 2